# Reflex Agents

Reflex AI Agents are the most fundamental class of intelligent agents. They are designed to react immediately to their environment without "thinking" about the future or analyzing complex consequences.

They function like a human reflex (e.g., blinking when an object flies toward your eye)—purely reactive and fast.

Reflex agents operate on a simple loop known as Condition-Action Rules (often called "If-Then" logic).

- **Condition**: The agent perceives a specific state in the environment.

- **Action**: The agent immediately executes a pre-assigned task associated with that state.

Key Characteristic: They do not plan, strategize, or simulate future outcomes. They only care about the "here and now."

In this notebook, we’ll explore two types of reflex agents through the classic Vacuum World scenario (two rooms: A and B):

1. **Simple Reflex Agent:** Reacts only to
the current room’s cleanliness.
2. **Model-Based Reflex Agent:** Maintains an internal model of the world to avoid redundant actions.

# **1. Simple Reflex Agent**

The Simple Reflex Agent follows a minimal set of rules:

- If the current room is dirty → clean it.
- If the current room is clean → move to the other room.

**Limitation**:
This agent has **no memory**. If both rooms are clean, it will endlessly oscillate between them because it cannot recall whether the other room has already been cleaned.

In [ ]:
import random

class SimpleReflexAgent:
    def __init__(self):
        # No internal memory (state) here!
        pass

    def decide(self, percept):
        """
        Receives percept: (current_location, is_dirty)
        Returns action: 'Suction on', 'Right', 'Left'
        """
        location, is_dirty = percept

        # Rule 1: If current spot is dirty, clean it.
        if is_dirty:
            return "Suction on"

        # Rule 2: If clean, move to the other room.
        elif location == 'A':
            return "Right"
        elif location == 'B':
            return "Left"

###Simulation: Simple Reflex Agent in Action
We initialize the environment with both rooms clean and run a short simulation to observe the agent’s behavior.

In [ ]:
# Environment State: Both rooms are initially clean
env_state = {'A': True, 'B': False} # False = Clean
agent_location = 'A'
agent = SimpleReflexAgent()

print("--- Simple Reflex Simulation ---")
for i in range(4):
    # 1. Get Percept
    is_dirty = env_state[agent_location]
    percept = (agent_location, is_dirty)

    # 2. Decide
    action = agent.decide(percept)

    print(f"Loc: {agent_location} | Dirty: {is_dirty} -> Action: {action}")

    # 3. Update Environment
    if action == "Suction on":
        env_state[agent_location] = False
    elif action == "Right":
        agent_location = 'B'
    elif action == "Left":
        agent_location = 'A'

--- Simple Reflex Simulation ---
Loc: A | Dirty: True -> Action: Suction on
Loc: A | Dirty: False -> Action: Right
Loc: B | Dirty: False -> Action: Left
Loc: A | Dirty: False -> Action: Right


Note: Notice how it keeps moving 'Right' and 'Left' even if everything is clean?
<br>It lacks the memory to know it's "Done".

# **2. Model-Based Reflex Agent**
To overcome the limitations of the simple reflex agent, we introduce a **Model-Based Reflex Agent**. This agent maintains an internal state—a dictionary representing what it knows about the world.

**Advantages**
- **Internal Memory:** Tracks the cleanliness status of both rooms.
- **Smarter Decisions:** If its internal model indicates both rooms are clean, it stops acting (i.e., returns `"NoOp (Job Done)"`).
- **Avoids Redundancy:** No endless back-and-forth when the task is complete.

In [ ]:
class ModelBasedReflexAgent:
    def __init__(self):
        # Internal State: Keeps track of the world status
        self.model = {
            'A': 'Unknown',
            'B': 'Unknown'
        }

    def update_state(self, location, is_dirty):
        """Updates internal model based on current percept"""
        self.model[location] = 'Dirty' if is_dirty else 'Clean'

    def decide(self, percept):
        location, is_dirty = percept

        # 1. Update Internal State first
        self.update_state(location, is_dirty)

        # 2. Check "World Model" to make a decision

        # Rule: If internal model says EVERYWHERE is clean, Stop.
        # (A simple reflex agent cannot do this)
        if all(status == 'Clean' for status in self.model.values()):
            return "NoOp (Job Done)"

        # Standard Cleaning Rules
        if is_dirty:
            return "Suction on"
        elif location == 'A':
            return "Right"
        elif location == 'B':
            return "Left"

###Simulation: Model-Based Reflex Agent in Action
Now we simulate the model-based agent in a scenario where Room B starts dirty. Notice how it cleans efficiently and then stops once both rooms are confirmed clean.

In [ ]:
env_state = {'A': False, 'B': True} # A is Clean, B is Dirty
agent_location = 'A'
agent = ModelBasedReflexAgent()

print("\n--- Model-Based Simulation ---")
for i in range(5):
    is_dirty = env_state[agent_location]
    percept = (agent_location, is_dirty)

    action = agent.decide(percept)

    print(f"Loc: {agent_location} | Dirty: {is_dirty} | Model: {agent.model} -> Action: {action}")

    if action == "NoOp (Job Done)":
        break # The agent is smart enough to stop!

    if action == "Suction on":
        env_state[agent_location] = False
    elif action == "Right":
        agent_location = 'B'
    elif action == "Left":
        agent_location = 'A'


--- Model-Based Simulation ---
Loc: A | Dirty: False | Model: {'A': 'Clean', 'B': 'Unknown'} -> Action: Right
Loc: B | Dirty: True | Model: {'A': 'Clean', 'B': 'Dirty'} -> Action: Suction on
Loc: B | Dirty: False | Model: {'A': 'Clean', 'B': 'Clean'} -> Action: NoOp (Job Done)


> Key Insight: The model-based agent uses its internal world model to recognize when the job is truly done, avoiding the infinite loop that traps the simple reflex agent.

### Conclusion

These examples highlight a foundational principle in intelligent agent design:  
**Even without explicit planning, maintaining an internal model of the world enables significantly smarter and more efficient behavior.**

- The **Simple Reflex Agent** reacts purely to the current percept, which can lead to redundant or infinite actions (e.g., oscillating between clean rooms).
- In contrast, the **Model-Based Reflex Agent** uses memory to track what it knows about the environment. This allows it to recognize when a task is truly complete and avoid unnecessary actions.

This idea—augmenting reactivity with state—scales directly into more sophisticated agent architectures you’ll explore later in the course, such as **goal-based agents** (which plan to achieve specific objectives) and **utility-based agents** (which optimize for preference or efficiency).

By evolving from reflexive to stateful reasoning, agents become not just responsive, but *purposeful*.